In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## This notebook implements parts of speech (POS) tagging using Hidden Markov Model (HMM). Implementation of Viterbi Algorithm is also shown later.

## Say words = w1....wN and tags = t1..tN ,then,
## P(tags | words) is_proportional_to  product P(ti| t{i-1}) * P(wi | ti)
## To find the best tag sequence for a given sequence of words,we want to find the tag sequence that has the maximum P(tags | words)


In [7]:
import nltk
from nltk.corpus import brown

In [10]:
brown.tagged_sents()

[[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')], [('The', 'AT'), ('jury', 'NN'), ('further', 'RBR'), ('said', 'VBD'), ('in', 'IN'), ('term-end', 'NN'), ('presentments', 'NNS'), ('that', 'CS'), ('the', 'AT'), ('City', 'NN-TL'), ('Executive', 'JJ-TL'), ('Committee', 'NN-TL'), (',', ','), ('which', 'WDT'), ('had', 'HVD'), ('over-all', 'JJ'), ('charge', 'NN'), ('of', 'IN'), ('the', 'AT'), ('election', 'NN'), (',', ','), ('``', '``'), ('deserves', 'VBZ'), ('the', 'AT'), ('praise', 'NN'), ('and', 'CC'), ('thanks', 'NNS'), ('of', 'IN'), ('the', 'AT'), ('City', 'NN-TL'), ('of', 'IN-TL'), ('Atlant

# Now, we will estimate P(wi|ti) from corpus data by calculating Maximum Likelihood Estimation MLE: P(wi | ti)=count(wi,ti)/count(ti)

In [8]:
# list of all the unique tags from the corpus

brown_word_tags=[]

#Manually adding the start and the end tag
for brown_sent in brown.tagged_sents():
    brown_word_tags.append(('START','START'))
    
    for words,tag in brown_sent:
        brown_word_tags.extend([(tag[:2],words)])
        
    brown_word_tags.append(('END','END'))
    

In [11]:
#Getting the continuous frequency distribution for the words which are tagged
cfd_tag_words=nltk.ConditionalFreqDist(brown_word_tags)

In [12]:
#Getting the conditional probability distribution
cpd_tag_words=nltk.ConditionalProbDist(cfd_tag_words,nltk.MLEProbDist)

In [26]:
print("The probability of an adjective (JJ) being 'smart' is", cpd_tag_words["JJ"].prob("smart"))
print("The probability of a verb (VB) being 'try' is", cpd_tag_words["VB"].prob("try"))

The probability of an adjective (JJ) being 'smart' is 0.00027780092785509904
The probability of a verb (VB) being 'try' is 0.0010790559555256297


# Estimating P(ti | t{i-1}) from corpus data using Maximum Likelihood Estimation (MLE):
# P(ti | t{i-1}) = count(t{i-1}, ti) / count(t{i-1})

In [22]:
brown_tags=[]
for tag, words in brown_word_tags:
    brown_tags.append(tag)

In [23]:
#make conditional frequency distribution: count(t{i-1} ti)
cfd_tags=nltk.ConditionalFreqDist(nltk.bigrams(brown_tags))

In [24]:
# make conditional probability distribution, using maximum likelihood estimate: P(ti | t{i-1})
cpd_tags=nltk.ConditionalProbDist(cfd_tags,nltk.MLEProbDist)

In [25]:
print('The probability of DT occuring after NN is : ', cpd_tags["NN"].prob("DT"))
print('The probability of VB occuring after NN is : ', cpd_tags["NN"].prob("VB"))

The probability of DT occuring after NN is :  0.0018349509874933604
The probability of VB occuring after NN is :  0.0646359290427087



## The probability of the tag sequence "PP VB NN" for the word sequence "I love food"?
## P(START) * P(PP|START) * P(I | PP) * P(VB | PP) * P(love | VB) * P(TO | VB) * P(food | NN) * P(END | VB)

In [28]:
prob_tagsequence = cpd_tags["START"].prob("PP") * cpd_tag_words["PP"].prob("I") * \
                   cpd_tags["PP"].prob("VB") * cpd_tag_words["VB"].prob("love") * \
                   cpd_tags["VB"].prob("NN") * cpd_tag_words["PP"].prob("food") * \
                   cpd_tags["NN"].prob("END")

In [29]:
print("The probability of sentence 'I love food' having the tag sequence 'START PP VB PP END' is : ", prob_tagsequence)

The probability of sentence 'I love food' having the tag sequence 'START PP VB PP END' is :  0.0


In [30]:
# Viterbi:
# If we have a word sequence, what is the best tag sequence?
#
# The method above lets us determine the probability for a single tag sequence.
# But in order to find the best tag sequence, we need the probability
# for _all_ tag sequence.
# What Viterbi gives us is just a good way of computing all those many probabilities
# as fast as possible.


distinct_brown_tags=set(brown_tags)

In [32]:
sample_sentences=["I","love","spicy","food"]
len_sample_sentence=len(sample_sentences)

In [35]:
# for each step i in 1 .. sentlen,
# store a dictionary
# that maps each tag X
# to the probability of the best tag sequence of length i that ends in X


viterbi_tags={}
viterbi_backpointer={}

for tag in distinct_brown_tags:
    if tag=="START":
        continue
    viterbi_tags[tag]=cpd_tags["START"].prob(tag)*cpd_tag_words[tag].prob(sample_sentences[0])
    viterbi_backpointer[tag]="START"

In [37]:
# for each step i in 1 .. sentlen,
# store a dictionary
# that maps each tag X
# to the probability of the best tag sequence of length i that ends in X



viterbi_main=[]
backpointer_main=[]

viterbi_main.append(viterbi_tags)
backpointer_main.append(viterbi_backpointer)

current_best=max(viterbi_tags.keys(),key=lambda tag: viterbi_tags[tag])



In [39]:
print("Word", "'" + sample_sentences[0] + "'", "current best two-tag sequence:", viterbi_backpointer[current_best], current_best)


Word 'I' current best two-tag sequence: START PP


In [44]:


for index in range(1,len_sample_sentence):
    curr_viterbi={}
    curr_backpointer={}
    prev_viterbi=viterbi_main[-1]
    
    for brown_tag in distinct_brown_tags:
        
        if brown_tag != "START":
            # if this tag is X and the current word is w, then
            # find the previous tag Y such that
            # the best tag sequence that ends in X
            # actually ends in Y X
            # that is, the Y that maximizes
            # prev_viterbi[ Y ] * P(X | Y) * P( w | X)
            # The following command has the same notation
            # that you saw in the sorted() command.
            prev_best = max(prev_viterbi.keys(),
                                key=lambda prevtag: \
                                    prev_viterbi[prevtag] * cpd_tags[prevtag].prob(brown_tag) * cpd_tag_words[brown_tag].prob(
                                        sample_sentences[index]))

            curr_viterbi[brown_tag] = prev_viterbi[prev_best] * \
                                cpd_tags[prev_best].prob(brown_tag) * cpd_tag_words[brown_tag].prob(sample_sentences[index])
            curr_backpointer[brown_tag] = prev_best

    current_best = max(curr_viterbi.keys(), key=lambda tag: curr_viterbi[tag])
    print("Word", "'" + sample_sentences[index] + "'", "current best two-tag sequence:", curr_backpointer[current_best], current_best)


    viterbi_main.append(curr_viterbi)
    backpointer_main.append(curr_backpointer)


Word 'love' current best two-tag sequence: PP NN
Word 'spicy' current best two-tag sequence: VB JJ
Word 'food' current best two-tag sequence: JJ NN


In [45]:
# now find the probability of each tag
# to have "END" as the next tag,
# and use that to find the overall best sequence



prev_viterbi = viterbi_main[-1]
prev_best = max(prev_viterbi.keys(),
                    key=lambda prev_tag: prev_viterbi[prev_tag] * cpd_tags[prev_tag].prob("END"))

prob_tag_sequence = prev_viterbi[prev_best] * cpd_tags[prev_best].prob("END")


best_tag_sequence = ["END", prev_best]
# invert the list of backpointers
backpointer_main.reverse()

# go backwards through the list of backpointers
# (or in this case forward, because we have inverter the backpointer list)
# in each case:
# the following best tag is the one listed under
# the backpointer for the current best tag
current_best_tag = prev_best
for backpointer in backpointer_main:
    best_tag_sequence.append(backpointer[current_best_tag])
    current_best_tag = backpointer[current_best_tag]


In [48]:
best_tag_sequence.reverse()

In [49]:
print("The sentence given is :")
for word in sample_sentences:
    print (word,"",)

The sentence given is :
I 
love 
spicy 
food 


In [50]:
print("The best tag sequence using HMM for the given sentence is : ")


for best_tag in best_tag_sequence:
    print (best_tag, "",)

The best tag sequence using HMM for the given sentence is : 
START 
PP 
VB 
JJ 
NN 
END 


In [51]:
print("The probability of the best tag sequence printed above is given by : ", prob_tag_sequence)

The probability of the best tag sequence printed above is given by :  1.085507090627119e-18
